In [1]:
import pandas as pd 
import uuid as uid
import psycopg2 as pg
from dotenv import load_dotenv
import os
import psycopg2.extras
import numpy as np

## Итоги работы можно найти в конце ноутбука, в разделе "Итоги работы"

# Проектирование базы данных

In [2]:
customer = pd.read_excel('/Users/zond/Desktop/python/vs_code/ipnd_les/data/asset-v1_SkillFactory+MFTIDS+SEP2023+type@asset+block@customer_and_transaction__2_.xlsx', sheet_name='customer')
transaction = pd.read_excel('/Users/zond/Desktop/python/vs_code/ipnd_les/data/asset-v1_SkillFactory+MFTIDS+SEP2023+type@asset+block@customer_and_transaction__2_.xlsx', sheet_name='transaction')

## Краткий обзор

In [3]:
display(customer.head(2))
display(transaction.head(2))

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10


,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92


In [4]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            4000 non-null   int64 
 1   first_name             4000 non-null   object
 2   last_name              3875 non-null   object
 3   gender                 4000 non-null   object
 4   DOB                    3913 non-null   object
 5   job_title              3494 non-null   object
 6   job_industry_category  3344 non-null   object
 7   wealth_segment         4000 non-null   object
 8   deceased_indicator     4000 non-null   object
 9   owns_car               4000 non-null   object
 10  address                4000 non-null   object
 11  postcode               4000 non-null   int64 
 12  state                  4000 non-null   object
 13  country                4000 non-null   object
 14  property_valuation     4000 non-null   int64 
dtypes: int64(3), object(1

In [5]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    20000 non-null  int64         
 1   product_id        20000 non-null  int64         
 2   customer_id       20000 non-null  int64         
 3   transaction_date  20000 non-null  datetime64[ns]
 4   online_order      19640 non-null  object        
 5   order_status      20000 non-null  object        
 6   brand             19803 non-null  object        
 7   product_line      19803 non-null  object        
 8   product_class     19803 non-null  object        
 9   product_size      19803 non-null  object        
 10  list_price        20000 non-null  float64       
 11  standard_cost     19803 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 1.8+ MB


<div style="background-color: rgba(0, 128, 0, 0.7);">
    Тут мы видим, что сейчас данные таблицы имеют связь между продуктом и клиентом многие ко многим, поэтому мы обязательно должны оставить таблицу-прокладку, в нашем случае это транзакции. Также обе таблицы уже находятся в первой нормальной форме, потому что нет атрибутов, которые бы содержали составные значения. Но сейчас таблица не находится во второй нормальной форме, так как некоторые атрибуты в таблице не имеют прямой связи с ключом. Начнем с выделения информации по продукту. Также стоит обратить внимание на пропуски в таблице transaction.
</div>


## Разделение данных на таблицы

### Таблица transaction

In [6]:
# Отображаем транзакции, где бренд продукта не указан
print("Транзакции, где информация о бренде отсутствует, но list_price проставлен и 'product_id' равен 0")
display(transaction[transaction['brand'].isna()].head())


# Отображаем транзакции, где идентификатор продукта равен 0
print("Все транзакции с 'product_id' равным 0")
display(transaction[transaction['product_id'] == 0].head())

Транзакции, где информация о бренде отсутствует, но list_price проставлен и 'product_id' равен 0


,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
136,137,0,431,2017-09-23,False,Approved,NaN,NaN,NaN,NaN,1942.61,NaN
159,160,0,3300,2017-08-27,False,Approved,NaN,NaN,NaN,NaN,1656.86,NaN
366,367,0,1614,2017-03-10,False,Approved,NaN,NaN,NaN,NaN,850.89,NaN
406,407,0,2559,2017-06-14,True,Approved,NaN,NaN,NaN,NaN,710.59,NaN
676,677,0,2609,2017-07-02,False,Approved,NaN,NaN,NaN,NaN,1972.01,NaN


Все транзакции с 'product_id' равным 0


,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
34,35,0,2171,2017-08-20,False,Approved,Norco Bicycles,Road,medium,medium,544.05,376.84
39,40,0,2448,2017-11-28,True,Approved,Norco Bicycles,Road,medium,medium,544.05,376.84
54,55,0,3140,2017-09-18,False,Approved,Norco Bicycles,Standard,low,medium,363.01,290.41
60,61,0,1839,2017-02-24,False,Approved,OHM Cycles,Road,high,large,12.01,7.21
63,64,0,2000,2017-07-08,False,Approved,Trek Bicycles,Standard,medium,medium,499.53,388.72


In [7]:
# Создаем новый DataFrame 'product' с выбранными столбцами, связанными с продуктом.
product = transaction[['product_id', 'brand', 'product_class', 'product_line', 'product_size', 'standard_cost']]

# Устанавливаем значение -1 для 'product_id' в 'transaction', где 'brand' не указан (NaN).
transaction.loc[transaction['brand'].isna(), 'product_id'] = -1

# Удаляем столбцы, связанные с продуктом, из 'transaction', модифицируя исходный DataFrame.
transaction.drop(['brand', 'product_class', 'product_line', 'product_size', 'standard_cost'], axis=1, inplace=True)

product.drop_duplicates(inplace=True)

/var/folders/8y/5tr527m128gf8rlrnfzg1kfw0000gn/T/ipykernel_5952/1955431082.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product.drop_duplicates(inplace=True)


<div style="background-color: rgba(0, 128, 0, 0.7);">
Здесь, в "product" как отдельную таблицу, мы выносим все поля, которые напрямую связаны с продуктом: его бренд, размер, цену и т.д. Таким образом, мы уходим от транзитивных связей и переходим к третьей нормальной форме. Стоит сказать, что оставшиеся поля в таблице "transaction" имеют прямое отношение к транзакции, проведенной для клиента. И хотя мы не знаем точно, что содержит в себе поле "list_price", проведя обзор, который показан выше, мы можем заключить, что данное поле относится к транзакции, так как оно напрямую завязано на уникальный ключ. Если вы обратите внимание на вывод информации по датафрейму, то можно заметить, что именно в тех полях, которые мы отнесли к продукту, отсутствует информация, но вот с "list_price" такого нет, поэтому оставим его с транзакцией. Осталось только решить проблему, что "product_id" равен 0 и для конкретного продукта, и для транзакции с пропуском, что приведет к проблемам при объединении, так как будет объединяться неправильная информация. Поэтому присвоим ему неестественный ключ, поскольку поле с ID не должно иметь значение "null".
</div>

### Таблица customer

<div style="background-color: rgba(0, 128, 0, 0.7);">
Теперь займёмся таблицей с клиентом. Как уже говорилось ранее, она находится в первой нормальной форме, но имеет поля, которые не зависят от первичного ключа. Таким примером может быть страна; поэтому давайте вынесем адрес в отдельную таблицу. Но перед этим создадим ключ address_id и будем использовать специальную библиотеку.

</div>

In [8]:
# Генерируем уникальный идентификатор адреса (address_id) для каждого клиента
customer.loc[:, 'address_id'] = customer.apply(lambda x: uid.uuid1(), axis=1)

# Создаем новый DataFrame 'address', содержащий информацию об адресе, извлеченную из таблицы 'customer'
address = customer[['address_id', 'address', 'postcode', 'state', 'country']]

# Удаляем столбцы, связанные с адресом, из таблицы 'customer'
customer.drop(['address', 'postcode', 'state', 'country'], axis=1, inplace=True)


In [9]:
# Отображаем первые пять адресов для проверки
display(address['address'].head())

# Считаем и выводим размер DataFrame
print( f"Количество записей в DataFrame 'address': {address.shape[0]}")
# Считаем и выводим количество дублирующихся записей по полям 'postcode', 'state', 'country'
print(f"Количество дубликатов по почтовому индексу, штату и стране: {address.duplicated(['postcode', 'state', 'country']).sum()}")

# Считаем и выводим количество дублирующихся записей по полям 'address', 'postcode', 'state', 'country'
print(f"Количество дубликатов по полному адресу: {address.duplicated(['address', 'postcode', 'state', 'country']).sum()}")


0     060 Morning Avenue
1    6 Meadow Vale Court
2     0 Holy Cross Court
3    17979 Del Mar Point
4       9 Oakridge Court
Name: address, dtype: object

Количество записей в DataFrame 'address': 4000
Количество дубликатов по почтовому индексу, штату и стране: 3005
Количество дубликатов по полному адресу: 0


<div style="background-color: rgba(0, 128, 0, 0.7);">
Здесь мы видим, что в новой таблице всего лишь 1000 уникальных комбинаций 'postcode', 'state', 'country', поэтому возникает вопрос: а не стоит ли нам разделить данную таблицу на две? Тогда данные будут занимать меньше места. Но есть и другая сторона: будет ли удобно пользоваться таким разделением в будущем? Возможно, эти данные будут использоваться для рекомендательной системы, которая будет дробить города на сегменты, и тогда пользователям придётся часто обращаться к комбинации улицы и штата, поэтому оставим как есть. Также тут мы видим, что в поле адрес указывается некое число, но понять, номер дома это или нет, не получается, так как дома не нумеруют с нулевого. Также мы не получили дополнительных данных от заказчика, поэтому оставим так.
</div>



<div style="background-color: rgba(0, 128, 0, 0.7);">
Поля 'job_title', 'job_industry_category', 'wealth_segment', 'owns_car', 'property_valuation' напрямую не относятся к каждому customer_id; скорее, их комбинация будет уникальной для customer_id, что не соответствует 3 нф и 2 нф. Также будем стараться сэкономить место в хранилище и упростить работу будущих пользователей, поэтому вынесем их в отдельные таблицы. С должностью и индустрией дело обстоит следующим образом: здесь мы видим, что по всем вынесенным столбцам мы можем сжать таблицу до 1000 строк вместо 4000.

</div>

In [10]:
# Считаем и выводим размер DataFrame
print(f"Количество записей в DataFrame 'customer': {customer.shape[0]}")

# Считаем и выводим количество дублирующихся записей по полям 'job_title', 'job_industry_category'
print(f"Количество дубликатов по job_title и job_industry_category: {customer.duplicated(['job_title', 'job_industry_category']).sum()}")

Количество записей в DataFrame 'customer': 4000
Количество дубликатов по job_title и job_industry_category: 2951


In [11]:
# Извлекаем столбцы 'job_title' и 'job_industry_category' из таблицы 'customer' и создаём новую таблицу 'job_title'
job_title = customer[['job_title', 'job_industry_category']]

# Удаляем дубликаты в таблице 'job_title' для получения уникальных комбинаций должностей и отраслей
job_title.drop_duplicates(inplace=True)

# Сбрасываем индекс для очистки и подготовки к добавлению идентификатора 'job_id'
job_title.reset_index(inplace=True)

# Генерируем уникальный идентификатор 'job_id' для каждой комбинации должности и отрасли
job_title.loc[:, 'job_id'] = job_title.apply(lambda x: uid.uuid1(), axis=1)

# Объединяем исходную таблицу 'customer' с новой таблицей 'job_title' по столбцам 'job_title' и 'job_industry_category'
customer = pd.merge(customer, job_title, on=['job_title', 'job_industry_category'])

# Удаляем из таблицы 'customer' столбцы 'job_title', 'job_industry_category' и 'index' после объединения
customer.drop(['job_title', 'job_industry_category', 'index'], axis=1, inplace=True)


/var/folders/8y/5tr527m128gf8rlrnfzg1kfw0000gn/T/ipykernel_5952/992270426.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  job_title.drop_duplicates(inplace=True)
/var/folders/8y/5tr527m128gf8rlrnfzg1kfw0000gn/T/ipykernel_5952/992270426.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  job_title.loc[:, 'job_id'] = job_title.apply(lambda x: uid.uuid1(), axis=1)


<div style="background-color: rgba(0, 128, 0, 0.7);">
Ситуация с благосостоянием получается достаточно интересной: уникальных комбинаций совсем мало, и хранить их в большой таблице было бы лишним. Да и это бы не соответствовало третьей нормальной форме, так как нет прямой связи между каждым полем и customer_id; скорее, wealth_segment зависит от owns_car или property_valuation.
</div>

In [12]:
# Считаем и выводим количество дублирующихся записей по полям 'wealth_segment', 'owns_car', 'property_valuation'
print(f"Количество дубликатов по 'wealth_segment', 'owns_car' и 'property_valuation': {customer[['wealth_segment', 'owns_car', 'property_valuation']].duplicated().sum()}")

Количество дубликатов по 'wealth_segment', 'owns_car' и 'property_valuation': 3928


In [13]:
# Создаем подмножество данных 'wealth' из 'customer' с информацией о благосостоянии
wealth = customer[['wealth_segment', 'owns_car', 'property_valuation']]

# Удаляем дубликаты для создания уникальных записей
wealth.drop_duplicates(inplace=True)

# Сбрасываем индекс после удаления дубликатов
wealth.reset_index(inplace=True)

# Генерируем уникальные идентификаторы 'wealth_id' для каждой записи
wealth.loc[:, 'wealth_id'] = wealth.apply(lambda x: uid.uuid1(), axis=1)

# Объединяем таблицу 'customer' с таблицей 'wealth' по соответствующим столбцам
customer = pd.merge(customer, wealth, on=['wealth_segment', 'owns_car', 'property_valuation'])

# Удаляем из 'customer' столбцы, которые теперь представлены в 'wealth', и сброшенный индекс
customer.drop(['wealth_segment', 'owns_car', 'property_valuation', 'index'], axis=1, inplace=True)


/var/folders/8y/5tr527m128gf8rlrnfzg1kfw0000gn/T/ipykernel_5952/447628126.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wealth.drop_duplicates(inplace=True)
/var/folders/8y/5tr527m128gf8rlrnfzg1kfw0000gn/T/ipykernel_5952/447628126.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wealth.loc[:, 'wealth_id'] = wealth.apply(lambda x: uid.uuid1(), axis=1)


In [14]:
# Замена значений в столбце 'gender'
customer['gender'] = customer['gender'].replace({
    'Male': 'M',
    'Female': 'F',
    'Femal': 'F' # Предполагаемая опечатка исправлена на 'F'
})


### Итог

In [15]:
print("Первая запись из таблицы 'customer':")
display(customer.head(1))

print("Первая запись из таблицы 'address':")
display(address.head(1))

job_title.drop(['index'], axis=1, inplace=True)
print("Первая запись из таблицы 'job_title':")
display(job_title.head(1))

wealth.drop(['index'], axis=1, inplace=True)
print("Первая запись из таблицы 'wealth':")
display(wealth.head(1))

print("Первая запись из таблицы 'transaction':")
display(transaction.head(1))

print("Первая запись из таблицы 'product':")
display(product.head(1))


Первая запись из таблицы 'customer':


,customer_id,first_name,last_name,gender,DOB,deceased_indicator,address_id,job_id,wealth_id
0,1,Laraine,Medendorp,F,1953-10-12 00:00:00,N,9276abb4-c455-11ee-a2c4-4eac44f25764,927b628a-c455-11ee-a2c4-4eac44f25764,927ddb32-c455-11ee-a2c4-4eac44f25764


Первая запись из таблицы 'address':


,address_id,address,postcode,state,country
0,9276abb4-c455-11ee-a2c4-4eac44f25764,060 Morning Avenue,2016,New South Wales,Australia


Первая запись из таблицы 'job_title':


...

,wealth_segment,owns_car,property_valuation,wealth_id
0,Mass Customer,Yes,10,927ddb32-c455-11ee-a2c4-4eac44f25764


Первая запись из таблицы 'transaction':


,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,list_price
0,1,2,2950,2017-02-25,False,Approved,71.49


Первая запись из таблицы 'product':


,product_id,brand,product_class,product_line,product_size,standard_cost
0,2,Solex,medium,Standard,medium,53.62


## Схема базы

<!DOCTYPE html>
<html lang="en">
<head>
<style>
.center {
  display: block;
  margin-left: auto;
  margin-right: auto;
  width: 70%;
  height: 75%;
}
</style>
</head>


<image src="/Users/zond/Desktop/python/vs_code/ipnd_les/db_project/dbdiagram_0.png" alt="Текст с описанием картинки" class="center">

</html>

# Загрузка данных в бд

In [16]:
# Загрузка переменных окружения из файла keys.env
load_dotenv("keys.env")

# Теперь получим переменные окружения
dbname = os.environ.get('DBNAME')
user = os.environ.get('DB_USER')
password = os.environ.get('PASSWORD')
host = os.environ.get('HOST')
port = os.environ.get('PORT')

In [17]:
conn = pg.connect(database=dbname,
                        host=host,
                        user=user,
                        password=password,
                        port=port)

In [18]:
cursor = conn.cursor()

# Путь к файлу SQL
sql_file_path = 'create_tables.sql'

# Открыть и выполнить SQL скрипты из файла
with open(sql_file_path, 'r') as file:
    sql_script = file.read()
    # Разделение скрипта на отдельные команды
    commands = sql_script.split(';')
    for command in commands:
        if command.strip():  # Проверка на пустую строку
            cursor.execute(command)

# Зафиксировать изменения
conn.commit()

# Закрыть курсор и соединение
cursor.close()
conn.close()

In [19]:
customer['DOB'] = pd.to_datetime(customer['DOB'])
customer['DOB'].fillna(pd.Timestamp('now'), inplace=True)
transaction['online_order'].fillna(False, inplace=True)
job_title = job_title[['job_id', 'job_title', 'job_industry_category']]
wealth = wealth[['wealth_id', 'wealth_segment', 'owns_car', 'property_valuation']]

In [20]:
conn = pg.connect(database=dbname,
                        host=host,
                        user=user,
                        password=password,
                        port=port)
conn.autocommit = True  # Включение автокоммита для выполнения каждой команды немедленно
psycopg2.extras.register_uuid()  # Регистрация поддержки UUID для psycopg2

sql_file_path = 'insert_values.sql'

cursor = conn.cursor()  

# Открытие файла SQL и чтение его содержимого
with open(sql_file_path, 'r') as file:
    sql_script = file.read()
    # Разделение скрипта на отдельные команды для последовательного выполнения
    commands = sql_script.split(';')
    df_list  = [customer, address, job_title, wealth, transaction, product]  # Список DataFrame'ов для вставки
    for command, df in zip(commands, df_list):  
        if command.strip():  
            data_tuples = list(df.itertuples(index=False, name=None))  # Преобразование DataFrame в список кортежей
            insert_query = command
            psycopg2.extras.execute_values(
                cursor, insert_query, data_tuples, template=None, page_size=100
            ) 

# Закрытие курсора и соединения с базой данных
cursor.close()
conn.close()

# Итоги работы 

После выполнения всех действий, которые указаны выше, и **SQL** скриптов из двух файлов, мы получаем следующие
<!DOCTYPE html>
<html lang="en">
<head>
    <style>
        .center {
            display: block;
            margin-left: auto;
            margin-right: auto;
            width: 70%;
            height: 75%;
        }
    </style>
</head>
<body>
    <figure>
        <img src="/Users/zond/Desktop/python/vs_code/ipnd_les/db_project/dbdiagram_0.png" alt="Схема базы данных" class="center">
        <figcaption>Схема базы данных</figcaption>
    </figure>
    <figure>
        <img src="/Users/zond/Desktop/python/vs_code/ipnd_les/db_project/tables.png" alt="Таблицы в базе данных, которые можно увидеть через UI pgAdmin" class="center">
        <figcaption>Таблицы в базе данных, которые можно увидеть через UI pgAdmin</figcaption>
    </figure>
</body>
</html>

Давайте напишем несколько запрсов к бд и убедимся, что данные доступны


In [27]:
conn = pg.connect(database=dbname,
                        host=host,
                        user=user,
                        password=password,
                        port=port)

query = (""" 
SELECT first_name
	  ,gender
	  ,EXTRACT(YEAR FROM AGE(CURRENT_DATE, "DOB")) AS YO
	  ,state
	  ,country
	  ,job_title
	  ,job_industry_category
	  ,wealth_segment
	  ,owns_car
	  ,property_valuation
	  ,brand
	  ,product_line

FROM customer 

JOIN address ON address.address_id = customer.address_id

JOIN job_title job ON job.job_id = customer.job_id

JOIN wealth ON wealth.wealth_id = customer.wealth_id

JOIN transaction trans ON trans.customer_id = customer.customer_id

JOIN product ON product.product_id = trans.product_id

WHERE 1=1
AND (CURRENT_DATE - "DOB")/360 < 50
AND (state = 'GDP' OR state = 'VIC')
AND job_industry_category LIKE '%IT%'
AND list_price > 1300
AND property_valuation > 8

LIMIT 10
       """)

pd.read_sql_query(query, conn)

/var/folders/8y/5tr527m128gf8rlrnfzg1kfw0000gn/T/ipykernel_5952/4052609317.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, conn)


,first_name,gender,yo,state,country,job_title,job_industry_category,wealth_segment,owns_car,property_valuation,brand,product_line
0,Natala,F,48.0,VIC,Australia,NaN,IT,Affluent Customer,No,9,Giant Bicycles,Standard
1,Natala,F,48.0,VIC,Australia,NaN,IT,Affluent Customer,No,9,OHM Cycles,Road
2,Stavro,M,42.0,VIC,Australia,Budget/Accounting Analyst IV,IT,Mass Customer,Yes,10,OHM Cycles,Standard
3,Stavro,M,42.0,VIC,Australia,Budget/Accounting Analyst IV,IT,Mass Customer,Yes,10,Trek Bicycles,Road
4,Stavro,M,42.0,VIC,Australia,Budget/Accounting Analyst IV,IT,Mass Customer,Yes,10,Trek Bicycles,Standard
5,Stavro,M,42.0,VIC,Australia,Budget/Accounting Analyst IV,IT,Mass Customer,Yes,10,Solex,Standard
6,Stavro,M,42.0,VIC,Australia,Budget/Accounting Analyst IV,IT,Mass Customer,Yes,10,OHM Cycles,Standard
7,Stavro,M,42.0,VIC,Australia,Budget/Accounting Analyst IV,IT,Mass Customer,Yes,10,Trek Bicycles,Road
8,Farris,U,0.0,VIC,Australia,Internal Auditor,IT,Mass Customer,Yes,10,Giant Bicycles,Standard
9,Farris,U,0.0,VIC,Australia,Internal Auditor,IT,Mass Customer,Yes,10,OHM Cycles,Road


In [29]:
conn.close()